# **UNIVERSIDAD TORCUATO DI TELLA**
## **MAESTRÍA EN ECONOMETRÍA**

---

### **TRABAJO PRÁCTICO DE ECONOMETRÍA**

- **Profesor:** González-Rozada, Martín  
- **Ayudante:** Lening, Iara  
- **Alumno:** Guzzi, David Alexander  (Legajo n°: 24H1970, DNI: 37.703.649)  

**Ciclo Lectivo:** Tercer Trimestre, 2024  

---

#### **1. PROPIEDADES DE MUESTRA FINITA DE FGLS (MCGE).**

Considere el siguiente modelo:

$$
\begin{equation}
y_i = \beta_0 + \beta_1 x_i + u_i, \quad i = 1, 2, \dots, 5N
\tag{1}
\end{equation}
$$

Donde:

- $\beta_0 = -3$
- $\beta_1 = 0.8$
- $u_j \sim N(0, \omega \cdot I_{N \times N})$, con la matriz $\Omega$ definida como:

$$
\Omega = \begin{bmatrix}
4 & 0 & 0 & 0 & 0 \\
0 & 9 & 0 & 0 & 0 \\
0 & 0 & 16 & 0 & 0 \\
0 & 0 & 0 & 25 & 0 \\
0 & 0 & 0 & 0 & 36
\end{bmatrix}
$$

- $x_j \sim U[1, 50]$.

In [2]:
# Importación de librerías.
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import t

# Semilla para la generación de números aleatorios.
np.random.seed(3649)

# Configuración de pandas (4 decimales).
pd.set_option('display.float_format', '{:.4f}'.format)

In [3]:
def fgls_estimation(x: np.ndarray, y: np.ndarray) -> tuple[float, float, float]:

    """
    Realiza una estimación de Feasible Generalized Least Squares (FGLS para un modelo lineal 
    con errores heterocedásticos.

    Parámetros:
    x : array-like
        Variable regresora.
    y : array-like
        Variable de respuesta.

    Retorna:
    beta0_fgls : float
        Intercepto estimado.
    beta1_fgls : float
        Coeficiente estimado para el regresor.
    se_beta1_fgls : float
        Error estándar del coeficiente estimado para el regresor.
    """

    # 1. Estimar modelo inicial y ~ x por OLS y obtener las estimaciones de los parámetros del modelo.
    X = np.column_stack((np.ones_like(x), x)) 
    beta_ols = np.linalg.inv(X.T @ X) @ X.T @ y 

    # 2. Calcular los residuos del modelo, elevarlos al cuadrado y obtener el logaritmo de los mismos.
    u_hat = y - X @ beta_ols
    u_hat2 = u_hat ** 2  
    log_u_hat2 = np.log(u_hat2)

    # 3. Dada la forma funcional de la heterocedasticidad de White para este modelo, sigma2 ~ x + x2, 
    # se estima por OLS usando u_hat2 como proxy de sigma2.
    X_aux = np.column_stack((x, x**2))
    gamma_hat = np.linalg.inv(X_aux.T @ X_aux) @ X_aux.T @ log_u_hat2
    
    # 4. Usar las estimaciones de la regresión auxiliar y obtener las varianzas ajustadas (no consistentes).
    log_sigma2_hat = X_aux @ gamma_hat
    sigma2_hat = np.exp(log_sigma2_hat)

    # 5. Transformar las variables de la forma funcional de la heterocedasticidad de White dividiéndolas por sigma2_hat y estimar por OLS.
    X_aux2 = np.column_stack((x / sigma2_hat,  x**2 / sigma2_hat))
    u_hat2_over_sigma2_hat = u_hat2 / sigma2_hat
    gamma_hat2 = np.linalg.inv(X_aux2.T @ X_aux2) @ X_aux2.T @ u_hat2_over_sigma2_hat

    # 6. Usar las estimaciones de la segunda regresión auxiliar y obtener las varianzas ajustadas (consistentes).
    sigma2_tilde = np.maximum(X_aux @ gamma_hat2, 1e-6) #Se asegura positividad de las varianzas. 
    omega_tilde_inv = np.linalg.inv(np.diag(sigma2_tilde)) #Se construye matriz omega inversa.  
    
    # 7. Aplicar GLS en la regresión de y ~ x, utilizando como ponderador la matriz omega estimada.
    # Obs.: También se podría usar uno sobre la raíz cuadrada de sigma2 tilde como ponderador en la regresión de y ~ x.
    
    # Se obtienen las estimaciones de los parámetros.
    beta_fgls = np.linalg.inv(X.T @ omega_tilde_inv @ X) @ X.T @ omega_tilde_inv @ y
    
    # Se obtienen las estimaciones de los errores estándar de los parámetros.
    residuals_fgls = y - X @ beta_fgls
    s2_fgls = (residuals_fgls.T @ omega_tilde_inv @ residuals_fgls) / ( X.shape[0] - X.shape[1])
    var_beta_fgls = s2_fgls * np.linalg.inv(X.T @ omega_tilde_inv @ X)
    se_beta1_fgls = np.sqrt(np.diag(var_beta_fgls))[1]   
    
    return beta_fgls[0], beta_fgls[1], se_beta1_fgls

In [4]:
def simulate_fgls(n_samples: int, n_observations_list: list[int], beta_0_true: float, beta_1_true: float, beta1_H0: float) -> pd.DataFrame:
    
    """
    Simula la estimación de FGLS para distintos tamaños de muestra y evalúa la validez de un test de hipótesis.

    Parameters:
    n_samples : int
        Número de simulaciones a realizar para cada tamaño de muestra.
    n_observations_list : list of int
        Lista con los distintos tamaños de muestra a evaluar.
    beta_0_true : float
        Valor verdadero del intercepto en la simulación.
    beta_1_true : float
        Valor verdadero del coeficiente del regresor en la simulación.
    beta1_H0 : float
        Valor hipotético de beta_1 bajo H0 para el test de hipótesis.

    Returns:
    results_df : pandas.DataFrame
        DataFrame con estadísticas de los coeficientes estimados y tasas de rechazo del test de hipótesis.
        Columnas:
        - 'n_observations': Tamaño de la muestra.
        - 'mean_beta0': Media de las estimaciones de beta_0.
        - 'median_beta0': Mediana de las estimaciones de beta_0.
        - 'std_beta0': Desviación estándar de las estimaciones de beta_0.
        - 'mean_beta1': Media de las estimaciones de beta_1.
        - 'median_beta1': Mediana de las estimaciones de beta_1.
        - 'std_beta1': Desviación estándar de las estimaciones de beta_1.
        - 'test_size_1pct': Proporción de rechazos de H0 al 1% de significancia.
        - 'test_size_5pct': Proporción de rechazos de H0 al 5% de significancia.
    """

    
    results = []
    for n_obs in n_observations_list:
        sample_results = []
        
        for i in range(n_samples):
            # Generación de datos.
            x = np.random.uniform(1, 50, n_obs) # x ~ U[1, 50]
            u = np.random.normal(scale=x) # u ~ N(0, omega), con varianza heterocedástica dependiendo de x.
            y = beta_0_true + beta_1_true * x + u
            
            # Estimación FGLS.
            beta_0_hat, beta_1_hat, se_beta1_hat = fgls_estimation(x, y)
            
            # Test de hipótesis para beta_1 = 0.8.
            t_stat = (beta_1_hat - beta1_H0) / se_beta1_hat
            p_value = 2 * (1 - t.cdf(abs(t_stat), df=n_obs - 2))
            reject_1pct = p_value < 0.01 # Rechazar H0 al 1% de significancia.
            reject_5pct = p_value < 0.05 # Rechazar H0 al 5% de significancia.
            
            sample_results.append([beta_0_hat, beta_1_hat, reject_1pct, reject_5pct])
        
        # Convertir a DataFrame.
        df_results = pd.DataFrame(sample_results, columns=['beta_0_hat', 'beta_1_hat', 'reject_1pct', 'reject_5pct'])
        
        # Calcular estadísticas.
        mean_beta0 = df_results['beta_0_hat'].mean()
        mean_beta1 = df_results['beta_1_hat'].mean()
        median_beta0 = df_results['beta_0_hat'].median()
        median_beta1 = df_results['beta_1_hat'].median()
        std_beta0 = df_results['beta_0_hat'].std()
        std_beta1 = df_results['beta_1_hat'].std()
        test_1pct = df_results['reject_1pct'].mean() #Equivale a la proporción de rechazos de las 5000 simulaciones al 1% de significancia.
        test_5pct = df_results['reject_5pct'].mean() #Equivale a la proporción de rechazos de las 5000 simulaciones al 5% de significancia.
        
        results.append([n_obs, mean_beta0, median_beta0, std_beta0, mean_beta1, median_beta1, std_beta1, test_1pct, test_5pct])
    
    results_df = pd.DataFrame(results, columns=['n_observations', 'mean_beta0', 'median_beta0', 'std_beta0', 'mean_beta1', 'median_beta1', 'std_beta1', 'test_1pct', 'test_5pct'])
    
    return results_df

##### **Punto 1a.** Para cada muestra se estima por FGLS los parámetros del modelo y se realiza un test de hipótesis para contrastar que H₀ : β₁ = 0.8. Se reporta el tamaño del test al 1% y 5%. Adicionalmente, la media, mediana y desvío estándar de las estimaciones de β₀ y β₁. Se repite el proceso con 5N = 10, 30, 100, 200, y 500 observaciones (puntos 2, 3, 4, 5, 6).

In [3]:
simulate_fgls(n_samples=5000, n_observations_list=[5, 10, 30, 100, 200, 500], beta_0_true=-3, beta_1_true=0.8, beta1_H0=0.8)


,n_observations,mean_beta0,median_beta0,std_beta0,mean_beta1,median_beta1,std_beta1,test_1pct,test_5pct
0,5,9.2582,-2.6233,893.4464,0.4592,0.7869,22.2380,0.1124,0.2204
1,10,18.4817,-2.9402,994.5864,0.3145,0.7957,23.2917,0.1118,0.1968
2,30,-14.9002,-2.9858,630.7914,1.0043,0.8017,13.5219,0.0750,0.1512
3,100,-4.3804,-3.0011,88.5079,0.8393,0.8005,3.1786,0.0578,0.1088
4,200,-2.9602,-2.9970,8.5433,0.8055,0.8000,2.8978,0.0480,0.0952
5,500,-2.9962,-2.9792,2.0976,0.8099,0.7984,1.7883,0.0370,0.0768


In [4]:
simulate_fgls(n_samples=5000, n_observations_list=[700, 1000, 1200, 1500, 2000], beta_0_true=-3, beta_1_true=0.8, beta1_H0=0.8)


,n_observations,mean_beta0,median_beta0,std_beta0,mean_beta1,median_beta1,std_beta1,test_1pct,test_5pct
0,700,-3.0200,-3.0115,1.8553,0.8116,0.7993,1.6837,0.0292,0.0690
1,1000,-3.0054,-2.9938,1.6094,0.8088,0.8000,1.4567,0.0236,0.0606
2,1200,-2.9859,-2.9945,1.1024,0.7919,0.8000,0.9931,0.0166,0.0584
3,1500,-2.9848,-3.0003,0.8170,0.7858,0.8009,0.7410,0.0146,0.0528
4,2000,-2.9913,-2.9926,0.2693,0.7976,0.7992,0.1837,0.0112,0.0472


In [5]:
simulate_fgls(n_samples=5000, n_observations_list=[5, 10, 30, 100, 200, 500], beta_0_true=-3, beta_1_true=0.8, beta1_H0=0.4)

,n_observations,mean_beta0,median_beta0,std_beta0,mean_beta1,median_beta1,std_beta1,test_1pct,test_5pct
0,5,9.2582,-2.6233,893.4464,0.4592,0.7869,22.2380,0.1182,0.2262
1,10,18.4817,-2.9402,994.5864,0.3145,0.7957,23.2917,0.1344,0.2470
2,30,-14.9002,-2.9858,630.7914,1.0043,0.8017,13.5219,0.2010,0.3766
3,100,-4.3804,-3.0011,88.5079,0.8393,0.8005,3.1786,0.6884,0.8616
4,200,-2.9602,-2.9970,8.5433,0.8055,0.8000,2.8978,0.9600,0.9864
5,500,-2.9962,-2.9792,2.0976,0.8099,0.7984,1.7883,0.9964,0.9974


In [6]:
simulate_fgls(n_samples=5000, n_observations_list=[5, 10, 30, 100, 200, 500], beta_0_true=-3, beta_1_true=0.8, beta1_H0=0)

,n_observations,mean_beta0,median_beta0,std_beta0,mean_beta1,median_beta1,std_beta1,test_1pct,test_5pct
0,5,1.7059,-3.0860,1245.7746,0.5090,0.7938,33.4404,0.1472,0.2760
1,10,75.3163,-2.9185,4386.8609,-0.9077,0.7989,98.9208,0.2142,0.3862
2,30,-11.9590,-3.0124,559.0582,0.9417,0.7998,11.6853,0.6186,0.7944
3,100,-3.3895,-2.9896,35.6139,0.7892,0.7953,3.2351,0.9848,0.9896
4,200,-3.0301,-2.9915,3.4799,0.8054,0.7994,2.1781,0.9950,0.9962
5,500,-3.0554,-3.0077,2.0244,0.8435,0.7993,1.7780,0.9968,0.9974
